In [131]:
import torch.nn as nn
import torch
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from data_preparator import *
from torch.utils.data import Dataset, DataLoader

class DAC(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(DAC, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size+1, self.embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size, 7)

    def forward(self, seq, lengths, gpu = True):
        print("Sequence shape: ", seq.shape)
        bs, seq_len = seq.size()
        print("Batch size: ", bs)
        self.hidden = self._init_hidden(bs, gpu)
        
        embeds = self.embedding(seq)
        embeds = pack_padded_sequence(embeds, lengths) #unpad
        
        gru_out, self.hidden = self.gru(embeds, self.hidden) #retorna o hidden_state de todos os timesteps
        
        gru_out, lenghts = pad_packed_sequence(gru_out) # faz o pad da sequencia para o tamanho maximo do batch
        
        print('GRU output(all timesteps)', gru_out.shape)
        print(gru_out)
        
        #Como é um problema de classificacao, vou usar a ultima camada hidden
        output = self.fc(self.hidden[-1])
        
        return F.log_softmax(output, dim=-1)
    
    def _init_hidden(self, batch_size, gpu):
        if gpu: return Variable(torch.zeros((1,batch_size,self.hidden_size)).cuda())
        else: return Variable(torch.zeros((1,batch_size,self.hidden_size)))
        return self.create_variable(hidden)

    def create_variable(self, tensor):
        # Do cuda() before wrapping with variable
        if torch.cuda.is_available():
            return Variable(tensor.cuda())
        else:
            return Variable(tensor)

In [132]:
padded, labels, vocab_size, x_lengs = prepare_dataset('./conversas_mexidas.csv')

In [133]:
hidden_size = 30

In [134]:
m = DAC(vocab_size, embedding_dim=20, hidden_size=30)

print(m)

DAC(
  (embedding): Embedding(2492, 20)
  (gru): GRU(20, 30, bidirectional=True)
  (fc): Linear(in_features=30, out_features=7, bias=True)
)


In [135]:
class DialogueDataset(Dataset):
    def __init__(self, encoded_dialogues, labels, x_lengs):
        self.len = encoded_dialogues.shape[0]
        self.x_data = encoded_dialogues
        self.y_data = torch.tensor(labels) #one-hot encoding
        self.x_lengs = x_lengs
        
    def __getitem__(self, index):
        xb = self.x_data[index]
        yb = self.y_data[index]
        lens = self.x_lengs[index]
        return xb, yb, lens
    
    def __len__(self):
        return self.len

In [136]:
dataset = DialogueDataset(padded, labels, x_lengs)
train_loader = DataLoader(dataset= dataset,
                         batch_size = 1,
                         shuffle=False)

In [137]:
it = iter(train_loader)

In [138]:
x, y, l = next(it)

In [139]:
outp = m(x, l)

Sequence shape:  torch.Size([1, 90])
Batch size:  1


ValueError: Expected `len(lengths)` to be equal to batch_size, but got 1 (batch_size=90).